# **Análisis y Predicción de Ventas en una Tienda de Retail**

In [ ]:
import pandas as pd

#Importar CSV
df = pd.read_csv('retail_sales_dataset.csv')
df.head(5)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


Crear nuevas columnas

In [ ]:
#Calcula el ingreso total por venta
df['Total ventas'] = df['Price per Unit'] * df['Quantity']

#clasificar la edad en rangos

def edad(i):
  if i >= 60:
    return 'Adulto Mayor'
  elif i <= 24:
    return 'Joven'
  else:
    return 'Adulto'

df['Categoria Edad'] = df['Age'].apply(edad)

df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Total ventas,Categoria Edad
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Adulto
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Adulto
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Adulto
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Adulto
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Adulto
...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,50,Adulto Mayor
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,90,Adulto
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,100,Joven
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,150,Adulto


Clasifica los datos

In [ ]:
#Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’)
bajo = df['Total Amount'].quantile(0.33)
alto =  df['Total Amount'].quantile(0.66)

def ventas(i):
  if i >= alto:
    return 'Alto'
  elif i <= bajo:
    return 'Bajo'
  else:
    return 'Medio'

df['Categoria Ventas'] = df['Total Amount'].apply(ventas)

df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Total ventas,Categoria Edad,Categoria Ventas
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Adulto,Medio
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Adulto,Alto
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Adulto,Bajo
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Adulto,Alto
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Adulto,Medio
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,50,Adulto Mayor,Bajo
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,90,Adulto,Bajo
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,100,Joven,Medio
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,150,Adulto,Medio


Agrupación y Agregación

In [ ]:
#Precio Promedio por Género y Producto
precio_genero = df.groupby('Gender')['Total Amount'].sum()

precio_genero

,Total Amount
Gender,
Female,232840
Male,223160


In [ ]:
#Total de Ventas por Producto y Rango de Edad
precio_edad = df.groupby('Categoria Edad')['Total Amount'].sum().reset_index()

precio_edad


,Categoria Edad,Total Amount
0,Adulto,336535
1,Adulto Mayor,44815
2,Joven,74650


In [ ]:
#Mínimo y Máximo Precio por Categoría de Producto:

minymax_producto = df.groupby('Product Category').agg(
    Maximo_Precio=('Price per Unit', 'max'),
    Minimo_Precio=('Price per Unit', 'min')
).reset_index()

minymax_producto

,Product Category,Maximo_Precio,Minimo_Precio
0,Beauty,500,25
1,Clothing,500,25
2,Electronics,500,25


Análisis Personalizado con apply

In [ ]:
#Calcula la desviación de cada venta respecto a la media de su grupo.
media_ventas = df.groupby('Product Category')['Total Amount'].mean().reset_index()
media_ventas.columns = ['Product Category', 'Media_Ventas']

df = df.merge(media_ventas, on='Product Category')

df['Desviacion'] = df['Total Amount'] - df['Media_Ventas']

columnas_interes = df[['Product Category', 'Total Amount', 'Desviacion']]

print(columnas_interes)

    Product Category  Total Amount  Desviacion
0             Beauty           150 -317.475570
1           Clothing          1000  556.752137
2        Electronics            30 -428.786550
3           Clothing           500   56.752137
4             Beauty           100 -367.475570
..               ...           ...         ...
995         Clothing            50 -393.247863
996           Beauty            90 -377.475570
997           Beauty           100 -367.475570
998      Electronics           150 -308.786550
999      Electronics           120 -338.786550

[1000 rows x 3 columns]
